<!--html_preserve-->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-130562131-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-130562131-1');
</script>
<!--/html_preserve-->


In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/uq1.jpg")



# Introduction{-}

This tutorial introduces how to extract and process text data from social media sites, web pages, or other documents for later analysis. The entire R markdown document for the present tutorial can be downloaded [here](https://slcladal.github.io/webcrawling.Rmd). This tutorial builds heavily on and uses materials from  [this tutorial](https://tm4ss.github.io/docs/Tutorial_2_Web_crawling.html) on web crawling and scraping using R by Andreas Niekler and Gregor Wiedemann [see @WN17]. [The tutorial](https://tm4ss.github.io/docs/index.html) by Andreas Niekler and Gregor Wiedemann is more thorough, goes into more detail than this tutorial, and overs many more very useful text mining methods. An alternative approach for web crawling and scraping would be to use the `RCrawler` package [@khalil2017rcrawler] which is not introduced here thought (inspecting the `RCrawler` package and its functions is, however, also highly recommended). For a more in-depth introduction to web crawling in scraping, @miner2012practical is a very useful introduction.

The automated download of HTML pages is called *Crawling*. The extraction of the textual data and/or metadata (for example, article date, headlines, author names, article text) from the HTML source code (or the DOM document object model of the website) is called *Scraping* [see @olston2010web]. 

## Preparation and session set up{-}

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/IntroR_workshop.html). For this tutorials, we need to install certain *packages* from an R *library* so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# clean current workspace
rm(list=ls(all=T))
# set options
options(stringsAsFactors = F)
# install libraries
install.packages(c("rvest", "readtext"))


## Getting started{-}

For web crawling and scraping, we use the package `rvest` and to extract text data from various formats such as PDF, DOC, DOCX and TXT files with the `readtext` package. The tasks described int his section consist of:

1. Download a single web page and extract its content

2. Extract links from a overview page and extract articles

3. Extract text data from PDF and other formats on disk

# Crawl single webpage{-}

As a first step, we will download a single web page from *The Guardian* and extract text together with relevant metadata such as the article date. Let's define the URL of the article of interest and load the `rvest` package, which provides very useful functions for web crawling and scraping.


In [ ]:
url <- "https://www.theguardian.com/world/2017/jun/26/angela-merkel-and-donald-trump-head-for-clash-at-g20-summit"
require("rvest")


A convenient method to download and parse a webpage is provided by the function `read_html` which accepts a URL as its main argument. The function downloads the page and interprets the html source code as an HTML or XML object. 



In [ ]:
html_document <- read_html(url)



HTML and XML objects are a structured representation of HTML/XML source code, which allows to extract single elements (headlines e.g. `<h1>`, paragraphs `<p>`, links `<a>`, ...), their attributes (e.g. `<a href="http://...">`) or text wrapped in between elements (e.g. `<p>my text...</p>`). Elements can be extracted in XML objects with XPATH-expressions. 

XPATH (see [here]( https://en.wikipedia.org/wiki/XPath)) is a query language that parses XML-tree structures and we use it to select the headline element from the HTML page.

The following xpath expression queries for first-order-headline elements `h1`, anywhere in the tree `//` which fulfill a certain condition `[...]`, namely that the `class` attribute of the `h1` element must contain the value `content__headline`.

The next expression uses R pipe operator %>%, which takes the input from the left side of the expression and passes it on to the function ion the right side as its first argument. The result of this function is either passed onto the next function, via a pipe (`%>%`) or it is assigned to variable, if it is the last operation in a pipe. Our pipe takes the `html_document` object, passes it to the `html_node` function, which extracts the first node fitting the given xpath expression. The resulting node object is passed to the `html_text` function which extracts the text wrapped in the `h1`-element.


In [ ]:
title_xpath <- "//h1[contains(@class, 'content__headline')]"
title_text <- html_document %>%
  html_node(xpath = title_xpath) %>%
  html_text(trim = T)


Let's see, what the title_text contains:



In [ ]:
cat(title_text)



Now we modify the xpath expressions, to extract the article info, the paragraphs of the body text and the article date. Note that there are multiple paragraphs in the article. To extract not only the first, but all paragraphs we utilize the `html_nodes` function and glue the resulting single text vectors of each paragraph together with the `paste0` function.



In [ ]:
intro_xpath <- "//div[contains(@class, 'content__standfirst')]//p"
intro_text <- html_document %>%
  html_node(xpath = intro_xpath) %>%
  html_text(trim = T)
cat(intro_text)
body_xpath <- "//div[contains(@class, 'content__article-body')]//p"
body_text <- html_document %>%
  html_nodes(xpath = body_xpath) %>%
  html_text(trim = T) %>%
  paste0(collapse = "\n")


Now, let's inspect the first 150 elements of the text body.



In [ ]:
cat(substr(body_text, 0, 150))



We now extract the date from the html document.



In [ ]:
date_xpath <- "//time"
date_object <- html_document %>%
  html_node(xpath = date_xpath) %>%
  html_attr(name = "datetime") %>%
  as.Date()
cat(format(date_object, "%Y-%m-%d"))


The variables `title_text`, `intro_text`, `body_text` and `date_object` now contain the raw data for any subsequent text processing.


# Following links{-}

Usually, we do not want download a single document, but a series of documents. In our second exercise, we want to download all Guardian articles tagged with *Angela Merkel*. Instead of a tag page, we could also be interested in downloading results of a site-search engine or any other link collection. The task is always two-fold: First, we download and parse the tag overview page to extract all links to articles of interest: 


In [ ]:
url <- "https://www.theguardian.com/world/angela-merkel"
html_document <- read_html(url)


Second, we download and scrape each individual article page. For this, we extract all `href`-attributes from `a`-elements fitting a certain CSS-class. To select the right contents via XPATH-selectors, you need to investigate the HTML-structure of your specific page. Modern browsers such as Firefox and Chrome support you in that task by a function called "Inspect Element" (or similar), available through a right-click on the page element.



In [ ]:
links <- html_document %>%
  html_nodes(xpath = "//div[contains(@class, 'fc-item__container')]/a") %>%
  html_attr(name = "href")


Now, `links` contains a list of `r length(links)` hyperlinks to single articles tagged with Angela Merkel. 



In [ ]:
head(links, 3)



But stop! There is not only one page of links to tagged articles. If you have a look on the page in your browser, the tag overview page has several more than 60 sub pages, accessible via a paging navigator at the bottom. By clicking on the second page, we see a different URL-structure, which now contains a link to a specific paging number. We can use that format to create links to all sub pages by combining the base URL with the page numbers.



In [ ]:
page_numbers <- 1:3
base_url <- "https://www.theguardian.com/world/angela-merkel?page="
paging_urls <- paste0(base_url, page_numbers)
# View first 3 urls
head(paging_urls, 3)


Now we can iterate over all URLs of tag overview pages, to collect more/all links to articles tagged with Angela Merkel. We iterate with a for-loop over all URLs and append results from each single URL to a vector of all links.



In [ ]:
all_links <- NULL
for (url in paging_urls) {
  # download and parse single ta overview page
  html_document <- read_html(url)
  # extract links to articles
  links <- html_document %>%
    html_nodes(xpath = "//div[contains(@class, 'fc-item__container')]/a") %>%
    html_attr(name = "href")
  
  # append links to vector of all links
  all_links <- c(all_links, links)
}


An effective way of programming is to encapsulate repeatedly used code in a specific function. This function then can be called with specific parameters, process something and return a result. We use this here, to encapsulate the downloading and parsing of a Guardian article given a specific URL. The code is the same as in our exercise 1 above, only that we combine the extracted texts and metadata in a data.frame and wrap the entire process in a function-Block.



In [ ]:
scrape_guardian_article <- function(url) {
  
  html_document <- read_html(url)
  
  title_xpath <- "//h1[contains(@class, 'content__headline')]"
  title_text <- html_document %>%
    html_node(xpath = title_xpath) %>%
    html_text(trim = T)
  
  intro_xpath <- "//div[contains(@class, 'content__standfirst')]//p"
  intro_text <- html_document %>%
    html_node(xpath = intro_xpath) %>%
    html_text(trim = T)
  
  body_xpath <- "//div[contains(@class, 'content__article-body')]//p"
  body_text <- html_document %>%
    html_nodes(xpath = body_xpath) %>%
    html_text(trim = T) %>%
    paste0(collapse = "\n")
  
  date_xpath <- "//time"
  date_text <- html_document %>%
    html_node(xpath = date_xpath) %>%
    html_attr(name = "datetime") %>%
    as.Date()
  
  article <- data.frame(
    url = url,
    date = date_text,
    title = title_text,
    body = paste0(intro_text, "\n", body_text)
  )
  
  return(article)
  
}


Now we can use that function `scrape_guardian_article` in any other part of our script. For instance, we can loop over each of our collected links. We use a running variable i, taking values from 1 to `length(all_links)` to access the single links in `all_links` and write some progress output.



In [ ]:
all_articles <- data.frame()
for (i in 1:length(all_links)) {
  cat("Downloading", i, "of", length(all_links), "URL:", all_links[i], "\n")
  article <- scrape_guardian_article(all_links[i])
  # Append current article data.frame to the data.frame of all articles
  all_articles <- rbind(all_articles, article)
}


In [ ]:
all_articles <- data.frame()
for (i in 1:10) {
  if (i < 4) {
    cat("Downloading", i, "of", length(all_links), "URL:", all_links[i], "\n")
  }
  article <- scrape_guardian_article(all_links[i])
  # Append current article data.frame to the data.frame of all articles
  all_articles <- rbind(all_articles, article)
}


We now inspect the first three articles and save the data to our disc.



In [ ]:
head(all_articles, 3)
# Write articles to disk
write.csv2(all_articles, file = "data/guardian_merkel.csv")


The last command write the extracted articles to a CSV-file in the data directory for any later use.

# Read various file formats{-}

In case you have already a collection of text data files on your disk, you can import them into R in a very convenient provided by the `readtext` package. The package depends on some other programs or libraries in your system, to provide extraction for Word- and PDF-documents. So there may be some hurdles to install the package.

But once it is successfully installed, it allows for very easy extraction of text data from various file formats. Fist, we request a list of files in the directory to extract text from. For demonstration purpose, we provide in `data/documents` a random selection of various text formats.


In [ ]:
data_files <- list.files(path = "data/documents", full.names = T, recursive = T)
# View first file paths
head(data_files, 3)


The `readtext` function from the package with the same name, detects automatically file formats of the given files list and extracts the content into a data.frame. The parameter `docvarsfrom` allows you to set metadata variables by splitting path names. In our example, `docvar3` contains a source type variable derived from the sub folder name. 



In [ ]:
require(readtext)
extracted_texts <- readtext(data_files, docvarsfrom = "filepaths", dvsep = "/")
# View first rows of the extracted texts
head(extracted_texts)
# View beginning of tenth extracted text
substr(trimws(extracted_texts$text[10]) , 0, 100)


Again, the `extracted_texts` can be written by `write.csv2` to disk for later use.

# Citation & Session Info {-}

Schweinberger, Martin. 2020. *Web Crawling and Scraping using R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/webcrawling.html (Version 2020.12.03).


In [ ]:
@manual{schweinberger2020webc,
  author = {Schweinberger, Martin},
  title = {Web Crawling and Scraping using R},
  note = {https://slcladal.github.io/webcrawling.html},
  year = {2020},
  organization = "The University of Queensland, School of Languages and Cultures},
  address = {Brisbane},
  edition = {2020/12/203}
}


In [ ]:
sessionInfo()



# References {-}

***

[Main page](https://slcladal.github.io/index.html)

***
